# Texas Tow Trucks

We're going to scrape some [tow trucks in Texas](https://www.tdlr.texas.gov/tools_search/).

## Import your imports

In [152]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

## Search for the TLDR Number `006179570C`, and scrape the information on that company

Using [license information system](https://www.tdlr.texas.gov/tools_search/), find information about the tow truck number above, displaying the

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

If you can't figure a 'nice' way to locate something, your two last options might be:

- **Find a "parent" element, then dig inside**
- **Find all of a type of element** (like we did with `td` before) and get the `[0]`, `[1]`, `[2]`, etc
- **XPath** (inspect an element, Copy > Copy XPath)

These kinds of techniques tend to break when you're on other result pages, but... maybe not! You won't know until you try.

> - *TIP: When you use xpath, you CANNOT use double quotes or Python will get confused. Use single quotes.*
> - *TIP: You can clean your data up if you want to, or leave it dirty to clean later*
> - *TIP: The address part can be tough, but you have a few options. You can use a combination of `.split` and list slicing to clean it now, or clean it later in the dataframe with regular expressions. Or other options, too, probably*

In [153]:
driver = webdriver.Chrome()
driver.get('https://www.tdlr.texas.gov/tools_search/')

In [154]:
button = driver.find_element_by_xpath('//*[@id="mcrbutton"]')
button.click()

In [155]:
TLDR = driver.find_element_by_xpath('//*[@id="mcrdata"]')
TLDR.send_keys('006179570C')

In [156]:
submit = driver.find_element_by_xpath('//*[@id="submit3"]')
submit.click()

In [157]:
block = driver.find_element_by_id('t1')
tables = block.find_elements_by_tag_name('table')

#The business name
#Owner/operator
#Phone number
block1 = tables[2]
tds = block1.find_elements_by_tag_name('td')
biz_name = tds[2].text.split(':')[1]
owner = tds[4].text.split(':')[1]
phone = tds[6].text.split(':')[1]

print(biz_name,owner,phone)
    

#License status (Active, Expired, Etc)
#Physical address
block2 = tables[3]
status = block2.find_element_by_tag_name('font').text
address = block2.find_elements_by_tag_name('td')[3].text.split('\n\n')[-1].split(':')[1]

print(status,address)


   B.D. SMITH TOWING    BRANDT SMITH / OWNER    8173330706
Expired 
13619 BRETT JACKSON RD.
FORT WORTH, TX. 76179


# Adapt this to work inside of a single cell

Double-check that it works. You want it to print out all of the details.

In [158]:
block = driver.find_element_by_id('t1')
tables = block.find_elements_by_tag_name('table')

#The business name
#Owner/operator
#Phone number
block1 = tables[2]
tds = block1.find_elements_by_tag_name('td')
biz_name = tds[2].text.split(':')[1][3:]
owner = tds[4].text.split(':')[1][3:]
phone = tds[6].text.split(':')[1][3:]


    

#License status (Active, Expired, Etc)
#Physical address
block2 = tables[3]
status = block2.find_element_by_tag_name('font').text
address = block2.find_elements_by_tag_name('td')[3].text.split('\n\n')[-1].split(':')[1][1:]

print(biz_name,owner,phone,status,address,sep='\n')

B.D. SMITH TOWING
BRANDT SMITH / OWNER
8173330706
Expired
13619 BRETT JACKSON RD.
FORT WORTH, TX. 76179


# Using .apply to find data about SEVERAL tow truck companies

The file `trucks-subset.csv` has information about the trucks, we'll use it to find the pages to scrape.

### Open up `trucks-subset.csv` and save it into a dataframe

In [159]:
import pandas as pd
df = pd.read_csv('trucks-subset.csv')
df

,TDLR Number
0,006507931C
1,006179570C
2,006502097C


## Go through each row of the dataset, displaying the URL you will need to scrape for the information on that row

You don't have to actually use the search form for each of these - look at the URL you're on, it has the number in it!

For example, one URL might look like `https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006495492C`.

- *TIP: Use .apply and a function*
- *TIP: Unlike the Yelp example, you'll need to build this URL from pieces*
- *TIP: You probably don't want to `print` unless you're going to fix it for the next question 
- *TIP: pandas won't showing you the entire url! Run `pd.set_option('display.max_colwidth', -1)` to display aaaalll of the text in a cell*

In [160]:
def get_url(TDLRnum):
    url = 'https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber='+TDLRnum
    return url

In [161]:
pd.set_option('display.max_colwidth', -1)

In [162]:
df.apply(get_url)

,TDLR Number
0,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C


### Save this URL into a new column of your dataframe, called `url`

- *TIP: Use a function and `.apply`*
- *TIP: Be sure to use `return`*

In [163]:
df['url']=df.apply(get_url)

In [164]:
df

,TDLR Number,url
0,006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C


In [165]:
df['url']

0    https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1    https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2    https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C
Name: url, dtype: object

## Go through each row of the dataset, printing out information about each tow truck company.

Now will be **scraping** inside of your function.

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

Just print it out for now.

- *TIP: use .apply*
- *TIP: You'll be using the code you wrote before, but converted into a function*
- *TIP: Remember how the TDLR Number is in the URL? You don't need to do the form submission if you don't want!*
- *TIP: Make sure you adjust any variables so you don't scrape the same page again and again*

In [166]:
def get_truck_info(TDLRnum):
    url = 'https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber='+TDLRnum
    driver.get(url)
    
    block = driver.find_element_by_id('t1')
    tables = block.find_elements_by_tag_name('table')

    #The business name
    #Owner/operator
    #Phone number
    block1 = tables[2]
    tds = block1.find_elements_by_tag_name('td')
    biz_name = tds[2].text.split(':')[1][3:]
    owner = tds[4].text.split(':')[1][3:]
    phone = tds[6].text.split(':')[1][3:]

    #License status (Active, Expired, Etc)
    #Physical address
    block2 = tables[3]
    status = block2.find_element_by_tag_name('font').text
    address = block2.find_elements_by_tag_name('td')[3].text.split('\n\n')[-1].split(':')[1][1:]
    
    print(biz_name,owner,phone,status,address,sep='\n')

In [167]:
df['TDLR Number'].apply(get_truck_info)

AUGUSTUS E SMITH
AUGUSTUS EUGENE SMITH / OWNER
9032276464
Active
103 N MAIN ST
BONHAM, TX. 75418
B.D. SMITH TOWING
BRANDT SMITH / OWNER
8173330706
Expired
13619 BRETT JACKSON RD.
FORT WORTH, TX. 76179
BARRY MICHAEL SMITH
BARRY MICHAEL SMITH / OWNER
8066544404
Active
4501 W CEMETERY RD
CANYON, TX. 79015


0    None
1    None
2    None
Name: TDLR Number, dtype: object

## Scrape the following information for each row of the dataset, and save it into new columns in your dataframe.

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

It's basically what we did before, but using the function a little differently.

- *TIP: Same as above, but you'll be returning a `pd.Series` and the `.apply` line is going to be a lot longer*
- *TIP: Save it to a new dataframe!*
- *TIP: Make sure you change your `df` variable names correctly if you're cutting and pasting - there are a few so it can get tricky*

In [168]:
def get_truck_info(num):
    url = 'https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber='+num
    driver.get(url)
    
    block = driver.find_element_by_id('t1')
    tables = block.find_elements_by_tag_name('table')

    #The business name
    #Owner/operator
    #Phone number
    block1 = tables[2]
    tds = block1.find_elements_by_tag_name('td')
    biz_name = tds[2].text.split(':')[1][3:]
    owner = tds[4].text.split(':')[1][3:]
    phone = tds[6].text.split(':')[1][3:]

    #License status (Active, Expired, Etc)
    #Physical address
    block2 = tables[3]
    status = block2.find_element_by_tag_name('font').text
    address = block2.find_elements_by_tag_name('td')[3].text.split('\n\n')[-1].split(':')[1][1:]
    
    return pd.Series({
        'Business Name':biz_name,
        'Owner/Operator':owner,
        'Phone Number':phone,
        'Liscense Status':status,
        'Physical Address':address
    })

In [169]:
df_truck=df['TDLR Number'].apply(get_truck_info)

In [170]:
df_truck

,Business Name,Owner/Operator,Phone Number,Liscense Status,Physical Address
0,AUGUSTUS E SMITH,AUGUSTUS EUGENE SMITH / OWNER,9032276464,Active,"103 N MAIN ST\nBONHAM, TX. 75418"
1,B.D. SMITH TOWING,BRANDT SMITH / OWNER,8173330706,Expired,"13619 BRETT JACKSON RD.\nFORT WORTH, TX. 76179"
2,BARRY MICHAEL SMITH,BARRY MICHAEL SMITH / OWNER,8066544404,Active,"4501 W CEMETERY RD\nCANYON, TX. 79015"


### Save your dataframe as a CSV named `tow-trucks-extended.csv`

In [171]:
df_truck.to_csv('tow-trucks-extended.csv',index = False)

### Re-open your dataframe to confirm you didn't save any extra weird columns

In [172]:
pd.read_csv('tow-trucks-extended.csv')

,Business Name,Owner/Operator,Phone Number,Liscense Status,Physical Address
0,AUGUSTUS E SMITH,AUGUSTUS EUGENE SMITH / OWNER,9032276464,Active,"103 N MAIN ST\nBONHAM, TX. 75418"
1,B.D. SMITH TOWING,BRANDT SMITH / OWNER,8173330706,Expired,"13619 BRETT JACKSON RD.\nFORT WORTH, TX. 76179"
2,BARRY MICHAEL SMITH,BARRY MICHAEL SMITH / OWNER,8066544404,Active,"4501 W CEMETERY RD\nCANYON, TX. 79015"


## Process the entire `tow-trucks.csv` file

We just did it on a short subset so far. Now try it on all of the tow trucks. **Save as the same filename as before**

In [174]:
all_truck = pd.read_csv('tow-trucks.csv')
all_truck

,TDLR Number
0,006507931C
1,006179570C
2,006502097C
3,006494912C
4,0649468VSF
5,006448786C
6,0648444VSF
7,0651667VSF
8,006017767C
9,006495492C


In [175]:
df_all_truck = all_truck['TDLR Number'].apply(get_truck_info)

In [176]:
df_all_truck

,Business Name,Owner/Operator,Phone Number,Liscense Status,Physical Address
0,AUGUSTUS E SMITH,AUGUSTUS EUGENE SMITH / OWNER,9032276464,Active,"103 N MAIN ST\nBONHAM, TX. 75418"
1,B.D. SMITH TOWING,BRANDT SMITH / OWNER,8173330706,Expired,"13619 BRETT JACKSON RD.\nFORT WORTH, TX. 76179"
2,BARRY MICHAEL SMITH,BARRY MICHAEL SMITH / OWNER,8066544404,Active,"4501 W CEMETERY RD\nCANYON, TX. 79015"
3,HEATH SMITH,HEATH A SMITH / OWNER,940-552-0687,Expired,"1529 WILBARGER ST\nVERNON, TX. 76384"
4,HEATH SMITH,HEATH A SMITH / OWNER,9405520687,Expired,"1529 WILBARGER ST\nVERNON, TX. 76384"
5,HYSMITH AUTOMOTIVE,WILLIAM THOMAS HYSMITH / PRESIDENT,ASHLEY ERIN HYSMITH / TREASURER,Active,"1210 US 380 BYPASS\nGRAHAM, TX. 76450"
6,HYSMITH AUTOMOTIVE & TRUCK REPAIR INC,HYSMITH ERIN ASHLEY / TREASURER,WILLIAM THOMAS HYSMITH / PRESIDENT,Suspended,"927 LOVING HWY\nGRAHAM, TX. 76450"
7,HYSMITH AUTOMOTIVE & TRUCK REPAIR INC,WILLIAM THOMAS HYSMITH / PRESIDENT,ASHLEY ERIN HYSMITH / TREASURER,Active,"1210 380 BYPASS\nGRAHAM, TX. 76450"
8,JEFF & WENDY SMITH,JEFF SMITH / PARTNER,WENDY SMITH / PARTNER,Suspended,"10842 FM 2138 N\nJACKSONVILLE, TX. 75766"
9,JEFF SMITH,JEFFREY JOHN SMITH / OWNER,8324354670,Active,"4338 HARVEY RD\nCROSBY, TX. 77532"


In [177]:
df_all_truck.to_csv('tow-trucks-extended.csv',index = False)

In [178]:
pd.read_csv('tow-trucks-extended.csv')

,Business Name,Owner/Operator,Phone Number,Liscense Status,Physical Address
0,AUGUSTUS E SMITH,AUGUSTUS EUGENE SMITH / OWNER,9032276464,Active,"103 N MAIN ST\nBONHAM, TX. 75418"
1,B.D. SMITH TOWING,BRANDT SMITH / OWNER,8173330706,Expired,"13619 BRETT JACKSON RD.\nFORT WORTH, TX. 76179"
2,BARRY MICHAEL SMITH,BARRY MICHAEL SMITH / OWNER,8066544404,Active,"4501 W CEMETERY RD\nCANYON, TX. 79015"
3,HEATH SMITH,HEATH A SMITH / OWNER,940-552-0687,Expired,"1529 WILBARGER ST\nVERNON, TX. 76384"
4,HEATH SMITH,HEATH A SMITH / OWNER,9405520687,Expired,"1529 WILBARGER ST\nVERNON, TX. 76384"
5,HYSMITH AUTOMOTIVE,WILLIAM THOMAS HYSMITH / PRESIDENT,ASHLEY ERIN HYSMITH / TREASURER,Active,"1210 US 380 BYPASS\nGRAHAM, TX. 76450"
6,HYSMITH AUTOMOTIVE & TRUCK REPAIR INC,HYSMITH ERIN ASHLEY / TREASURER,WILLIAM THOMAS HYSMITH / PRESIDENT,Suspended,"927 LOVING HWY\nGRAHAM, TX. 76450"
7,HYSMITH AUTOMOTIVE & TRUCK REPAIR INC,WILLIAM THOMAS HYSMITH / PRESIDENT,ASHLEY ERIN HYSMITH / TREASURER,Active,"1210 380 BYPASS\nGRAHAM, TX. 76450"
8,JEFF & WENDY SMITH,JEFF SMITH / PARTNER,WENDY SMITH / PARTNER,Suspended,"10842 FM 2138 N\nJACKSONVILLE, TX. 75766"
9,JEFF SMITH,JEFFREY JOHN SMITH / OWNER,8324354670,Active,"4338 HARVEY RD\nCROSBY, TX. 77532"
